In [2]:
import boto3
import configparser
import json

In [3]:
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get("default", "aws_access_key_id") 
access_key = config.get("default", "aws_secret_access_key")

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = int(config.get("DWH","DWH_NUM_NODES"))
DWH_NODE_TYPE = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")

In [3]:
iam = boto3.client('iam',
    aws_access_key_id=access_id,
    aws_secret_access_key=access_key,
    region_name='us-east-2'
)

In [6]:
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'}}],
                'Version': '2012-10-17'})
    )
    roleArn = dwhRole['Role']['Arn']
    config.set("DWH","IAM_ROLE_ARN", roleArn)
except Exception:
    print("Role already created")
    roleArn = config.get("DWH", "IAM_ROLE_ARN")

Role already created


In [8]:
redshift = boto3.client('redshift',
    region_name="us-east-2",
    aws_access_key_id=access_id,    
    aws_secret_access_key=access_key
)

In [9]:
response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
    NodeType = DWH_NODE_TYPE,
    NumberOfNodes = DWH_NUM_NODES,
    DBName = DWH_DB,
    ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
    MasterUsername = DWH_DB_USER,
    MasterUserPassword = DWH_DB_PASSWORD,
    IamRoles = [roleArn]
)

In [5]:
response = redshift.delete_cluster(
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    SkipFinalClusterSnapshot=True,
)

In [28]:
with open('dwh.cfg', 'w') as configfile:
    config.write(configfile)